# Introduction

This notebook will deviate from my standard notebook content of the Kaggle seriousness that is data mining, analytics or predictions with models. On the contrary I would like to make this kernel a fun and rather seemingly arbitrary one about a topic that has intrigued me since primary school - Fractals,  albeit hopefully not too arbitrary at the end.

Fractals can be thought of in the simplest terms as geometric objects that have a most peculiar and magical property, that of a self-similarity or recursive pattern in their structure when observed across different scales - i.e. the structure appears the same even when the object is magnified. Interestingly, fractals appear quite regularly in nature for example that of the Romanesco Brocolli to the shapes of coastlines. It is the latter topic that will be the focal point of investigation in this notebook, where I will examine how fractal-like are the shapes of coastlines of different countries (incidentally, it was calculating the length of the coastline of Great Britain that started Mandelbrot's treatise on Fractal Geometry). 

The outline of this kernel will therefore be structured as follows:
1. The Mandelbrot Fractal  
2. Loading in coastline pictures as Base64 encoding workaround
3. Fractal Dimension of various coastlines
4. Use in feature engineering

In [ ]:
# Loading in relevant packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.misc import imread
import base64
import codecs
import cv2 #Used for converting the coastline images to grayscale
from scipy import misc
%matplotlib inline   

## 1. The Mandelbrot fractal

To start things off, let us plot *the* famous Mandelbrot fractal pattern via the Mandelbrot set which is really just iterating through this quadratic equation:

$$
z_{n+1} = z_{n}^{2} + c 
$$


The Mandelbrot set is essentially the set of complex numbers c, for which the equation above does not diverge when the initial value of z is set to 0. Therefore defining and iterating through the equation allows us to plot the fractal as follows:

In [ ]:
def mandelbrot(n, m, itermax, xmin, xmax, ymin, ymax):
    ix, iy = np.mgrid[0:n, 0:m]
    x = np.linspace(xmin, xmax, n)[ix]
    y = np.linspace(ymin, ymax, m)[iy]
    c = x+complex(0,1)*y
    del x, y
    img = np.zeros(c.shape, dtype=int)
    ix.shape = n*m
    iy.shape = n*m
    c.shape = n*m
    z = np.copy(c)
    for i in range(itermax):
        if not len(z): break
        np.multiply(z, z, z)
        np.add(z, c, z)
        rem = abs(z)>2.0
        img[ix[rem], iy[rem]] = i+1
        rem = ~rem
        z = z[rem]
        ix, iy = ix[rem], iy[rem]
        c = c[rem]
    return img

plt.figure(figsize=(18,12))
import time
start = time.time()
I = mandelbrot(400, 400, 100, -2, .5, -1.25, 1.25)
print('Running time:', time.time()-start)
I[I==0] = 101
img = plt.imshow(I.T, origin='lower left', cmap='cubehelix_r')
axes = plt.gca()
# Hide grid lines
axes.grid(False)
# Hide axes ticks
axes.set_xticks([])
axes.set_yticks([])
plt.show()

   ## 2. Loading in coastline pictures via Base64 encoding

This section will briefly discuss a sort of hack or workaround that has helped me in loading pictures which I cannot obtain or find in any of the existing Kernel datasets. Furthermore if you feel that it is too much of a hassle to create a whole new dataset just to load your desired few pictures, you can use this trick ( In this case, I want to load in a handful of coastline images which I can't find ). 

This workaround is first obtain the picture you want, after which you obtain that picture's Base64 encoding by using any of the online converters (I used this [https://www.base64-image.de/](http://)). Having obtained the encoding, just copy and paste the encoding itself into the notebook and create the object which should store said encoding. In this case, I have store the encodings for the coastlines of three different countries - Singapore, Australia and Great Britain, but I have also made the input hidden as it is a very long string that would just take up unnecessary space. Just unhide it if you want to view the full code.

Finally via the codecs module, I can decode the Base64 encoding which allows one to reverse engineer the required image and to simultaneously convert it to grayscale via the cv2 module as follows:

In [ ]:
singapore = b'...'

In [ ]:
australia = b'...'

In [ ]:
gb = b'...'

Reading and decoding the images for the Singapore, Australian and Great British coastline and converting to grayscale so that we can carry out the latter analysis, we get to:

In [ ]:
# Generate the Mask for the Singapore coastline
f1 = open("singapore.png", "wb")
f1.write(codecs.decode(singapore,'base64'))
f1.close()
sing = imread("singapore.png")
sing = cv2.imread("singapore.png")
sing = cv2.cvtColor( sing, cv2.COLOR_RGB2GRAY )
# plt.imshow(sing)

Reading in the Australian coastline :

In [ ]:
# Generate the Mask for the Australian coastline
f1 = open("aus.png", "wb")
f1.write(codecs.decode(australia,'base64'))
f1.close()
aus = imread("aus.png")
aus = cv2.imread("aus.png")
aus = cv2.cvtColor( aus, cv2.COLOR_RGB2GRAY )
# plt.imshow(aus)

Loading in Great Britain's coastline

In [ ]:
# Generate the Mask for Great Britain's coastline
f1 = open("gb.png", "wb")
f1.write(codecs.decode(gb,'base64'))
f1.close()
gb = imread("gb.png")
gb = cv2.imread("gb.png")
gb = cv2.cvtColor( gb, cv2.COLOR_RGB2GRAY )
# plt.imshow(gb)

# 3. Fractal Dimension of coastlines (*and how similar they are*)

So now we arrive at the core section of this kernel and that would be to evaluate how self-similar or fractal-like is an object. Therefore much like standard geometrical objects where we utilise cartesian, polar or radial coordinates to help us make sense of the space occupied, for fractals we use the metric known as the Fractal Dimension.

The Fractal Dimension is basically a way to specifically the dimensions of a fractal (much like how a point is in 1D space, a line in 2D etc), but in a way which factors in the self-similarity. The definition can thus be given as the ratio of the number of self-similar pieces with the magnification factor in which the object can be broken into.

### Box counting to estimate the Fractal dimension

The most basic method of estimating the fractal dimension of an object is to use the method of box counting.
Let us first define our box counting function as follows:

In [ ]:
def boxcount(Z, k):
    S = np.add.reduceat(
        np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0),
                        np.arange(0, Z.shape[1], k), axis=1)
    return len(np.where((S > 0) & (S < k*k))[0])

With this box counting function, we can then define a function which calculates the fractal dimension as follows:

In [ ]:
def fractal_dimension(Z, threshold=0.9):
    Z = (Z < threshold)
    p = min(Z.shape)
    n = 2**np.floor(np.log(p)/np.log(2))
    n = int(np.log(n)/np.log(2))
    sizes = 2**np.arange(n, 1, -1)
    counts = []
    for size in sizes:
        counts.append(boxcount(Z, size))
    coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
    return -coeffs[0]

### 3a. Great Britain's coastlines

In [ ]:
Z = 1.0 - gb/255
sizes = 128, 64, 32
# sizes = 32, 16, 8
xmin, xmax = 0, Z.shape[1]
ymin, ymax = 0, Z.shape[0]
fig = plt.figure(figsize=(14, 11))
for i, size in enumerate(sizes):
    ax = plt.subplot(1, len(sizes), i+1, frameon=False)
    ax.imshow(1-Z, plt.cm.gray, interpolation="bicubic", vmin=0, vmax=1,
              extent=[xmin, xmax, ymin, ymax], origin="upper")
    ax.set_xticks([])
    ax.set_yticks([])
    for y in range(Z.shape[0]//size+1):
        for x in range(Z.shape[1]//size+1):
            s = (Z[y*size:(y+1)*size, x*size:(x+1)*size] > 0.25).sum()
            if s > 0 and s < size*size:
                rect = patches.Rectangle(
                    (x*size, Z.shape[0]-1-(y+1)*size),
                    width=size, height=size,
                    linewidth=.5, edgecolor='.25',
                    facecolor='.75', alpha=.25, color='darkblue')
                ax.add_patch(rect)

plt.tight_layout()
plt.show();
print("The Fractal dimension of the GB coastline is {}".format(fractal_dimension(Z, threshold=0.25)));

From the above plots which shows the variations in box size while approximating the shape of the coastline, it is apparent that the smaller the box size that we use, the greater the boxes tend to "hug" and hence better resolve the shape of the coastline.
### Fractal Dimension of Great Britain coastline : 1.24

### 3b. Australian Coastline

In [ ]:
Z = aus/255
sizes = 128, 64, 18
#sizes = 60, 16, 8
xmin, xmax = 0, Z.shape[1]
ymin, ymax = 0, Z.shape[0]
fig = plt.figure(figsize=(16, 13))
for i, size in enumerate(sizes):
    ax = plt.subplot(1, len(sizes), i+1, frameon=False)
    ax.imshow(1-Z, plt.cm.gray, interpolation="bicubic", vmin=0, vmax=1,
              extent=[xmin, xmax, ymin, ymax], origin="upper")
    ax.set_xticks([])
    ax.set_yticks([])
    for y in range(Z.shape[0]//size+1):
        for x in range(Z.shape[1]//size+1):
            s = (Z[y*size:(y+1)*size, x*size:(x+1)*size] > 0.25).sum()
            if s > 0 and s < size*size:
                rect = patches.Rectangle(
                    (x*size, Z.shape[0]-1-(y+1)*size),
                    width=size, height=size,
                    linewidth=.5, edgecolor='.25',
                    facecolor='.75', alpha=.25, color='darkblue')
                ax.add_patch(rect)

plt.tight_layout()
# plt.savefig("fractal-dimension.png")
plt.show()
print(fractal_dimension(Z, threshold=0.25))

### Fractal Dimension of Australian coastline : 1.15

### 3c. Singaporean Coastline

In [ ]:
Z = 1.0 - sing/255
#sizes = 128, 64, 32
sizes = 32, 16, 8
xmin, xmax = 0, Z.shape[1]
ymin, ymax = 0, Z.shape[0]
fig = plt.figure(figsize=(18, 14))
for i, size in enumerate(sizes):
    ax = plt.subplot(1, len(sizes), i+1, frameon=False)
    ax.imshow(1-Z, plt.cm.gray, interpolation="bicubic", vmin=0, vmax=1,
              extent=[xmin, xmax, ymin, ymax], origin="upper")
    ax.set_xticks([])
    ax.set_yticks([])
    for y in range(Z.shape[0]//size+1):
        for x in range(Z.shape[1]//size+1):
            s = (Z[y*size:(y+1)*size, x*size:(x+1)*size] > 0.25).sum()
            if s > 0 and s < size*size:
                rect = patches.Rectangle(
                    (x*size, Z.shape[0]-1-(y+1)*size),
                    width=size, height=size,
                    linewidth=.5, edgecolor='.25',
                    facecolor='.75', alpha=.25,color='darkblue')
                ax.add_patch(rect)

plt.tight_layout()
# plt.savefig("fractal-dimension.png")
plt.show()
print(fractal_dimension(Z, threshold=0.25))

### Fractal Dimension of Singapore coastline : 1.31

## The fractal dimension of the whole World's coastlines

So at this juncture, we should note the pretty remarkable result we have arrived at which is that having taken three separate and distinct islands - Great Britain, Singapore and Australia, their coastlines still show a very distinct self-similarity to each other given by the fact that the fractal dimensions of all three coastlines are within each other by a hair's breadth. 

This leads us to now ask the big question - can we also arrive at a roughly similar value in terms of the Fractal Dimension if we take our box counting technique and apply it to the coastline of the **ENTIRE** world map?

In [ ]:
# Load in the World map's Base64 encoding
world = b'iVBORw0KGgoAAAANSUhEUgAAA18AAAJVBAMAAADThFKIAAAACXBIWXMAAAsTAAALEwEAmpwYAAAKT2lDQ1BQaG90b3Nob3AgSUNDIHByb2ZpbGUAAHjanVNnVFPpFj333vRCS4iAlEtvUhUIIFJCi4AUkSYqIQkQSoghodkVUcERRUUEG8igiAOOjoCMFVEsDIoK2AfkIaKOg6OIisr74Xuja9a89+bN/rXXPues852zzwfACAyWSDNRNYAMqUIeEeCDx8TG4eQuQIEKJHAAEAizZCFz/SMBAPh+PDwrIsAHvgABeNMLCADATZvAMByH/w/qQplcAYCEAcB0kThLCIAUAEB6jkKmAEBGAYCdmCZTAKAEAGDLY2LjAFAtAGAnf+bTAICd+Jl7AQBblCEVAaCRACATZYhEAGg7AKzPVopFAFgwABRmS8Q5ANgtADBJV2ZIALC3AMDOEAuyAAgMADBRiIUpAAR7AGDIIyN4AISZABRG8lc88SuuEOcqAAB4mbI8uSQ5RYFbCC1xB1dXLh4ozkkXKxQ2YQJhmkAuwnmZGTKBNA/g88wAAKCRFRHgg/P9eM4Ors7ONo62Dl8t6r8G/yJiYuP+5c+rcEAAAOF0ftH+LC+zGoA7BoBt/qIl7gRoXgugdfeLZrIPQLUAoOnaV/Nw+H48PEWhkLnZ2eXk5NhKxEJbYcpXff5nwl/AV/1s+X48/Pf14L7iJIEyXYFHBPjgwsz0TKUcz5IJhGLc5o9H/LcL//wd0yLESWK5WCoU41EScY5EmozzMqUiiUKSKcUl0v9k4t8s+wM+3zUAsGo+AXuRLahdYwP2SycQWHTA4vcAAPK7b8HUKAgDgGiD4c93/+8//UegJQCAZkmScQAAXkQkLlTKsz/HCAAARKCBKrBBG/TBGCzABhzBBdzBC/xgNoRCJMTCQhBCCmSAHHJgKayCQiiGzbAdKmAv1EAdNMBRaIaTcA4uwlW4Dj1wD/phCJ7BKLyBCQRByAgTYSHaiAFiilgjjggXmYX4IcFIBBKLJCDJiBRRIkuRNUgxUopUIFVIHfI9cgI5h1xGupE7yAAygvyGvEcxlIGyUT3UDLVDuag3GoRGogvQZHQxmo8WoJvQcrQaPYw2oefQq2gP2o8+Q8cwwOgYBzPEbDAuxsNCsTgsCZNjy7EirAyrxhqwVqwDu4n1Y8+xdwQSgUXACTYEd0IgYR5BSFhMWE7YSKggHCQ0EdoJNwkDhFHCJyKTqEu0JroR+cQYYjIxh1hILCPWEo8TLxB7iEPENyQSiUMyJ7mQAkmxpFTSEtJG0m5SI+ksqZs0SBojk8naZGuyBzmULCAryIXkneTD5DPkG+Qh8lsKnWJAcaT4U+IoUspqShnlEOU05QZlmDJBVaOaUt2ooVQRNY9aQq2htlKvUYeoEzR1mjnNgxZJS6WtopXTGmgXaPdpr+h0uhHdlR5Ol9BX0svpR+iX6AP0dwwNhhWDx4hnKBmbGAcYZxl3GK+YTKYZ04sZx1QwNzHrmOeZD5lvVVgqtip8FZHKCpVKlSaVGyovVKmqpqreqgtV81XLVI+pXlN9rkZVM1PjqQnUlqtVqp1Q61MbU2epO6iHqmeob1Q/pH5Z/YkGWcNMw09DpFGgsV/jvMYgC2MZs3gsIWsNq4Z1gTXEJrHN2Xx2KruY/R27iz2qqaE5QzNKM1ezUvOUZj8H45hx+Jx0TgnnKKeX836K3hTvKeIpG6Y0TLkxZVxrqpaXllirSKtRq0frvTau7aedpr1Fu1n7gQ5Bx0onXCdHZ4/OBZ3nU9lT3acKpxZNPTr1ri6qa6UbobtEd79up+6Ynr5egJ5Mb6feeb3n+hx9L/1U/W36p/VHDFgGswwkBtsMzhg8xTVxbzwdL8fb8VFDXcNAQ6VhlWGX4YSRudE8o9VGjUYPjGnGXOMk423GbcajJgYmISZLTepN7ppSTbmmKaY7TDtMx83MzaLN1pk1mz0x1zLnm+eb15vft2BaeFostqi2uGVJsuRaplnutrxuhVo5WaVYVVpds0atna0l1rutu6cRp7lOk06rntZnw7Dxtsm2qbcZsOXYBtuutm22fWFnYhdnt8Wuw+6TvZN9un2N/T0HDYfZDqsdWh1+c7RyFDpWOt6azpzuP33F9JbpL2dYzxDP2DPjthPLKcRpnVOb00dnF2e5c4PziIuJS4LLLpc+Lpsbxt3IveRKdPVxXeF60vWdm7Obwu2o26/uNu5p7ofcn8w0nymeWTNz0MPIQ+BR5dE/C5+VMGvfrH5PQ0+BZ7XnIy9jL5FXrdewt6V3qvdh7xc+9j5yn+M+4zw33jLeWV/MN8C3yLfLT8Nvnl+F30N/I/9k/3r/0QCngCUBZwOJgUGBWwL7+Hp8Ib+OPzrbZfay2e1BjKC5QRVBj4KtguXBrSFoyOyQrSH355jOkc5pDoVQfujW0Adh5mGLw34MJ4WHhVeGP45wiFga0TGXNXfR3ENz30T6RJZE3ptnMU85ry1KNSo+qi5qPNo3ujS6P8YuZlnM1VidWElsSxw5LiquNm5svt/87fOH4p3iC+N7F5gvyF1weaHOwvSFpxapLhIsOpZATIhOOJTwQRAqqBaMJfITdyWOCnnCHcJnIi/RNtGI2ENcKh5O8kgqTXqS7JG8NXkkxTOlLOW5hCepkLxMDUzdmzqeFpp2IG0yPTq9MYOSkZBxQqohTZO2Z+pn5mZ2y6xlhbL+xW6Lty8elQfJa7OQrAVZLQq2QqboVFoo1yoHsmdlV2a/zYnKOZarnivN7cyzytuQN5zvn//tEsIS4ZK2pYZLVy0dWOa9rGo5sjxxedsK4xUFK4ZWBqw8uIq2Km3VT6vtV5eufr0mek1rgV7ByoLBtQFr6wtVCuWFfevc1+1dT1gvWd+1YfqGnRs+FYmKrhTbF5cVf9go3HjlG4dvyr+Z3JS0qavEuWTPZtJm6ebeLZ5bDpaql+aXDm4N2dq0Dd9WtO319kXbL5fNKNu7g7ZDuaO/PLi8ZafJzs07P1SkVPRU+lQ27tLdtWHX+G7R7ht7vPY07NXbW7z3/T7JvttVAVVN1WbVZftJ+7P3P66Jqun4lvttXa1ObXHtxwPSA/0HIw6217nU1R3SPVRSj9Yr60cOxx++/p3vdy0NNg1VjZzG4iNwRHnk6fcJ3/ceDTradox7rOEH0x92HWcdL2pCmvKaRptTmvtbYlu6T8w+0dbq3nr8R9sfD5w0PFl5SvNUyWna6YLTk2fyz4ydlZ19fi753GDborZ752PO32oPb++6EHTh0kX/i+c7vDvOXPK4dPKy2+UTV7hXmq86X23qdOo8/pPTT8e7nLuarrlca7nuer21e2b36RueN87d9L158Rb/1tWeOT3dvfN6b/fF9/XfFt1+cif9zsu72Xcn7q28T7xf9EDtQdlD3YfVP1v+3Njv3H9qwHeg89HcR/cGhYPP/pH1jw9DBY+Zj8uGDYbrnjg+OTniP3L96fynQ89kzyaeF/6i/suuFxYvfvjV69fO0ZjRoZfyl5O/bXyl/erA6xmv28bCxh6+yXgzMV70VvvtwXfcdx3vo98PT+R8IH8o/2j5sfVT0Kf7kxmTk/8EA5jz/GMzLdsAAAAgY0hSTQAAeiUAAICDAAD5/wAAgOkAAHUwAADqYAAAOpgAABdvkl/FRgAAAA9QTFRFAAAAAIAAubm5/4BB////1khhPwAALgNJREFUeNrtnWmO5DCORnNy6gBTQB0gQXwHGKAvUEXw/mfqH960UBIlb5LDRqO7OjMiMqxn7hT1Je810vX3612DF9h7vcDe6wX2AnuvF9h7vcBeYO/1AnuvF9gDgRER3gXpH9jMiZbrhdY/sPB6l2UwYC+x0YC9xEYDRiLCL7iBgL2CNiKw12EcCVhntPh9enLAuludV+BP8BL5ZGCvTU0Ca3uWT9Rar1E9OnBmOjW5xfP3eqGpwFD55J8PbJZgfvXizlyi+yYCn21peMlU4wXWZsHcd81J/wvW8pMFbW+2/pZg4JM9kN3lldkVIILwVWv4yUH0TqdDRISEZ+/tKj1Fy6OCF1ib13BHfp8/Uy3udDocM3Y1sA81ZBqwppWTydtmupQX0fLPDwZW/dCy8ya+6HvzpxaDFGBNT/sBq8Vs+du8+DcfWm6NgbUs/RGLZfrjRMcVyJlowADhGGCy477Z/LepdKHhCRlNMGNgV38D619mOgvYUN1GtwNj65/lg3nNgRxjrBA8AnaPgKFKFI9pQuEh/coQmNwiYKiRxc9u8QqAyS3Aao3dJ/dQ+sDu4QXDy2DixfQ5wHDLzdoeE7I5iLx92nMV5Arsnj/PFgEjy7WZOLgxOD9NX24SdsuftywmU+MF5614HDC6ixdKmS465EKjeHfsdNyjEOMlCxxHppOIjelful5iJxrRX96jeMV/iF37NxowyG1Rc2IdD+eVEbGRxGwGJrck1NSVcivJR/KKbo+HzJSsEnZHzjoHDEd5GxGTNcgeMre12LB7REzz07b0PZ1CLAzbBiM2AZsfvaPEpiITntZUOFYfekG1wTvp1apNwA7WcjUPbPzXzwS2y63sSsJyIkFVKm7dyEU171OYn6ETxydmCJztX3xL55EIwU7al8duRIwMGvxGYLu/yba+TERc43b6f39drtuJUfDt+HbZOy75y+5WseW/6rQpfGLSAzHVx7yRmeslHhFT8SJgxKtbBvsHIPw/1NUVPEF3ATtCK3twaANW32PjiGtvIkYkt4duf78O+ZtrLOrbsdLD6JgGjgaTrfmXrq/rgR0WLS+dBq4hyz6LSc/CbX6jl9gJwHjjg4SIrXe2sstY93AvUdcXxgTGESekcq9U4OUxpiGu0YDpgqXnw3n5H9OzyzTKNRIwXROCEsDM3p+MBIzGAeZ7h/D/EQebinf8CGA0DDBfmuCJmpfREbPxGsdJjB6w/oGx68xrEiZe9wbX3D+NdV3QGXMAMJ9XCIySGcdjqo2fpho1YHV/kgN3UEIHvxHXYBcuOgzFB8ZLEaoKGBJ6DOH3fzAvitKhVwDjlofE99/hdmV8ErAp6y2ni5gDrLFy4GtEiYGR2vHxSBHztjydDYzb8mMcAyMFGNOjJczZkXYyMR3YLo0YAVsyUvxsfegs5OUS1ho0+2ldKDXJR+vDazMd1LL/xs9KhWldVjk+X8Yu8xKr/zYHecEADI+aFRwlW8+1jwopFkzLUpnTvS+xysC5TrI1hZgA9glX99l69tWeUvnnzyKGvoGxzwRaOPdhxKhjYKx6gqEv+GFK8QJiBmBqTY7JBOyxEsY9A+NE52iczoicd36sIKk/5G5VItmAPVUjQm9/7cVLpLRCjFLy/GwBc4bB9BA4V3scBWB4oGSt4xZuyk21AVOkCMqvnglM76x8ArDnRWFLbgAxMBlFJa52KvYfn+ci6k+tDAUMkU7wzPNzgbkB6IUFscPceiII81JxfmjTjQZMRHadjHG1hLFnszK27mnArjVd5wALyzP8dGCQ4YAh4yg9hxfiAxH4Fvnab8NEA8ZPcTqme4EmUXyPgO0EJkjxekLYjPCeI/UxDjD2NIV3e7z8kJ9ksfQVGAfYqhHix5GXH/KzBEwDhnGASdgjtf0CYfPAEwXs/Cb6E5pwdGBTHDm+DYPFincKLNf9pgJjCreNPRQYugSW3WjN2jfnLsZTnm7C7pAxGzDYgfE6wBzPN2E3EDO1CKSeN1KBLXWVRwO7vnJZCUz/uWJ9SU0zPljC+hsOlkrC8AvsBqVoBCYFYHDVJM+jH4bWiRCrTqQOVSISP+ZYAEEEmczYo4Fx18ASJgxLdkrxGpmIR65gVgBDb8ASRmw6TyAM03gtP4NGHBdlFpuOgSVugFOuyHwbGNr1qCnidgkMRmAINyLxAcqJu9OIcteZA3lgs4MI1VFkXezWiUV0SB0T7gTojgTsLhHLAwNts1JsKUZFsnaJx1LG6FDCOgTGLjDlW/3+E/KjY4FteodfnVgGNj3Zaw055vX7t4j8+7Plr2gbd+k0mLavWaAZLyUHu4h1YsPW70KJbMfv37//TP/lJa8Z/sLiGFdjKgIIriWW5daZl+h+Gw3Yv9/L9SdYRCa9mbtmtTQuQnKpA4LCAW/cFzC3a0GLnVdev8PFZQn41V/6e+QUB6QQ3ZeVovRjw9zR5fDckd/O9Sd8MukkYGeYsdkFbjNmXQHbJCx6kIjOBobG39Xz8uOHlAQit0boDBhTDOxfDpiPqM2GXQQMeau5OlKUW6SunA7XG3JmO5wNjC1SccRl8WaR8RZ7BuZatH92lYiWDiq25D+OFTHirJ2rLO9eDywyYa6P75mwyE1EtLy10VNJKo+LxkAmEePMMded2LD10VF4+QKmAQtPTceRAmYlJpV/ilt0InfjdMTDUjZVmQdGDArbE9mYNLTygEV4LKrTJGKZ7ueLdWIOmO69skioESMjxrypfjYssETIcAAw3p4a4wdxzrOv7KC4sYDpB4hEJCVgiCUHRdVVOczUKoNS8TnIZTbTIibdAXOSZtNN+ioxzCWqNy7lZ1wcDxBlYAW9CZslswKTW3bItgFzHY7p/nyvnmF5/KW8ZDKdiWEzYsJgxUrFNTS2S2pOJzINAoy9KTeIgAnHMLjCaUOsHNnoti8dkEHEFJjN/IfZgnzIpfNu9gDzHiwFmGaAYPMFky+uCbO814pX+WSnTJIM3mFTAyjs4unM6XBdWCISxd8wjQWLG6DAe1OD4ISzI/7fTSCzZqovj7iOADY/hHAk7I9yr2ys+08h2+6kBbQ/rBW5IIq0szFcuNyBPwDYUmZxgf1mL2lYa+UPKW6BYUynbP7uTCA+GFWyPQsYAxj8jAe7wByffj7WDRUh7zGlEjCJzfJNehHro8fmAA99iJhRwuA5bhD8U6wYbB7DCcBqdCtEm7qfUdo+MIyiEsOv7ylFuB2kRQh8BjC02EJYXaMVtIwCLP7+jogJEwmLFQLOANY0ZgJW12iJncfwElVDjNDtwFaIuAEYmvwX1KQqJZcB7izToVyRY4/FfrnlGG5PBtUX0ND2tnyswM8D5iSC/C7tlOPGJ9SOua3XQ0rS5/iR0oVOtO7AVDXi/32tqSly+qsw90AQ63mn6sKWWVTq93tKusg5p3WCw43HcOuVVf8j8u+b/kcrePFSj9QNC9cWtmqsUQOxVJYRkzX2TwMaxa2PV52Jfoh+tHX3/sVFLXSIUsQe/Klv4B3yCb9vYjC3HkQBMCSaJKBlGo5Xitj1Yblnxgd2e8a+ERgvwNb8oiZgMTCoIV2KZEUukffRR/lXa3puDGCs3cQ3/Zr+OTkYKrDFdsdpIU9Lpt3R2qxUw6eshfR8A8NSERwQ2Pz/fr6IaJ4/quc5JiPmbV1SVzwXQFSKBxreDzHEatKFTmyUMA5jSyzEhKOEkXh7BlKJ1dTvqqNf3vn+pAMqPbgdjclfuFtaZ19KlGFFvO6gSB1r6q5Fo4jtBGbNVUsXVyswBK4FT8oxXC4Qb3s4k1KEQ4FV6sRiVMHBlxyzvOIW47HedOwVwuGEVBCei9DrJQTH8gI9DJjMtU1RI2Umt1oN3UmMTWWFkx+vOB/JK4jDhlWJUQEXJELxuHqw837dq0dUxpGKDUrK68xvlZpNF9tDNaCXuN3G16/5d5OQcZR7wvYB0JVe+EjMPrZx2VmHaBFRNuxu8ZP1cn/GvhHYGonRj7gTweIqB8L7TAGL+ixMxJCQHbYbJ+Pz4CgbDAjs65vARN9EvyYK7Nwf6wZK/ySkgNmIcbt5qnogPM9oPGA/9AMRXoEtfQHwQ01OTOjLBs2oMkbc7lC0Ars12dEIbAa1AZuzcZgzTbzYdJQ/Kw0snzIqKzbjfqTasJnvdBkbs/ULsB/6WSt7rkpcfUf1zoKESIARxhq+QU6wHxgU1X6nl58frIICMKav73XrOms2zNKHhUDuOKTJzcvOh+nE+OntDZiXkWGeHI05SKK/9OOFwzJ3cHC4hpnOOQlC0vQ6ohkYdksYcRQ235pcLALbzMEP/Wzf8utX9KxtKcYyMCHaolak3MSMKIhRTng/sGjiMd2Z+0gDi7zvb6Lv+ORVL/Sd228NwMR1CoJHxNBEIHNjFpeAYU9Waq3VkbYykK6BEX0TfSPwHThMLoXTSPNuJ/zhWjxFAsgJClYLOI+rRVGjQU85m3gpvaP3Zu/rgP1iLxzx3Y7pR0TCZmDzsm3ZEFkRSOKgJPa7D4sbOLGMPtF+anE5WFPn3CGw6I48G7YAkx/6cbIVIBY/kCp9um8hCIS1G1AdQxslSqzVFM/2WBP6UE9YwI0ZxYzTASUdFQP7Jvre7greSsMS2cF9WiFMvBZplNYaNPnmHrKaPshEqqbL1JSe8XYZOMA2v8E1YuwHctBVLvxVITAHKS7nn9zm6jnIangltlz2CkxZjVhmZmDwgEGWASkBHtdjcfxm8VKr0M5tmTfEg3dUKym5oUZ/LRJagrsEphoM5QPm7l/IlqGa7NKU+wjsoW+5IUtbgfegbJ6zS2zyH+HL/YlD0UHcT6G5DZj65b9+uccfuWXBRdhIO85uduoXkUn0dbh05v5er9J15rED6KlZygYM6eYL30/a1nwbWj8dtROroEDhslK9SIRLayEZZ/KCpVuKhgAWhyMxMe/BhJ65FfcclTi5mjNS7G1hOkW+1iMxMmqF7xO8LDDOKUQmbQKms9c5FYD7LMK+Wsm6gbxtEgSfwmsN0nIpXu6wvBJn6bJP2TLIeKawuX3F7CvrVTEVGAgLp5N4rX3lyGZDO0xNFYFJUIVdcx2uZ4FyQcrP2XM24Y7V5TjL20AITAYChryLGABzCmcoa0PH8Zs1HFLv4dCENYRf1cCCQoL2OI8NTLwypBXYlLtlNzuVbQ1cTy04Lfxa9ftSThgGWOQkFlJqTDEwQ0PGpOTAuQ0scOO6qeOQ+URezv5C5Hzo4YDp+NRew0z1ApPjl/Mqw84DyEki5t9ZvmLeIbC4GyabfHffaRWxZa6YA4zTxLg9h2g2YGE2tTedeBwwaMrSnlaQbOCG6PuckuRQi+LjACuoxEyyxuwn6p3QqUoH3P4bnKcSvS+SXJ/OVGLoipW+HhQfX+xtoAEwpeV3SaIsu6lxuk6cv8g4wKwaMajAVkVicd1FJlcwtmU8CdfyxUBnixhvPTz9JKeswFCo9Yc/MLqJGjAmXciwzv3AebFzWJDIpH+78xJhtWAKMKtfrxkx1s8RnfOUa4fOSaGY7z9x6uapO6eD7cBCfc5Ellw9iTe9VMnvw28RkCWTCBI5Ldnhe/Mp9dIfMImP5ot+D82n95NWhbqT1sOo+u5TlmP5wTkHcLsyj7BupCxNd8CUToxUsI98aTOuOK2bNuMmxjj+Wg2W8wxppbMzbJkKjE2O8/XARCszI+Vr6D4+k3NOnszuH7aoCqFnw6pXwlpZWM7npVsx6hWYqKGks4SWP8HkKn3kUEDZAq0FXVcUWTLA2JZLuANYEhe5PWDz3qMCtun+ImCgbMJJBXZqU0cgOomIegBglAI27+6ziKi20ZLLkVoABie3JUoeGI0BTH30RUTke9mZbtKPsZgUgYWCBD/FeH6CSh2v1KUN47S3tyUBrMAYKrCCiCndNt5JvnIyMM+IeSmzDoEx0jkmzgLTboj82M5Y2uKUXVv+LXIdsJr6xS3ASs1MaRumZRjbgFGhL+dwvZhOanBN/eKmOIyI9UYL595UL5EjUy06sLINy/zsjDxwOhi9Xb5MmQ5kDuSo+Esc3LQRGNI/3CaAnQpMqZ9DbrtsqanUNvH6L844DBi2vS040YZtjtfyu36B0QnA1niZbcDSPTlMZ4Vjqdvq4XQIY7Y+9eA3AzMfDZA2YecBKwHpdPxedkHQqMzXrSfUDgzRl+LrgE07FLsERgZg1AiMi2Yq88vzgVGy/DclqLsEVlg1bhMxtYGG64DxTcBI7j96Jb/dyLBqtSI27245Aph+Ps95OnGp5/UJjEtLuumleq+eicmsEzn9rJyXtEcqE3D7IZgpYGXTz0HOXm0dUAVM4gJyDTDEbzu+LKavyd20dgHzvIdQNpEEpp1GkF3xXOpXLgXGHQPzOwLViWn+ZMQon5VO/2tLnCkdI/cDPquMCf0pHgJYaqcyq+cm5AeDMc8CCLGaMWREDufVnQcDZllK6JMs888os172Nx8yD+ZrmnFSjn33wDizkqzcX941ntx6RQ2yDZg/0JJPBIZE6Pw4YG5SN/pcuMNUjPkp72w51run6BJi/QJjk7LygEGvycDT/vOENE66DyUrlurGEbrI7+gKGNS8b85/00rm0Woux0sJYR74yso6wwIM5ZDsZL+jJ2CsAcu1akbjdlGsyQTAiPcBYzqTWPfANJVYV/qA5GWGAVInwxq9Ds54/PIRvCZgkTFl2ypw3kFIWT0k2mmkFhiHC8yPN2Hy90uLB23eV2livPajQMK2QiiTpBoMk49PFMkd2mvfL7A5HkTo1Zee2EZg/qoumSpoKcy88w6lyeTxSvHvl4hAGLHPYR+yYfs1ExFDzdSv46v1HNhai8ppRD4+hEa/TgdBwlk0BiNeJYAQImKOe+Hn/e+SBAZS1aVatgZ/DLAgV2FxuiqB8WLEIs3Dkjn4PiXJep+BTqzNH+kYWJSXqjwz17LOyykzurmwGLHsT9l5MCRWu5AH2TDFReTjgSEHLDuJBaaPZ+cPgTTH5pnAKox3LbD80eRpYHr7by4GJKVAhGcCYzoL2HR4WslimCOEfNS+7IZy311tx4YARucBQ3gesBlY/JdyczCXxAljHSdWToyOkusIgVENMNQCk+xuyYxORBwKlEpnMmWa2d2bW0cMgzgdmXXbB4wKwGB2O9JtCxIlLuHuSMPTgNGZElY8RDv9rMD8h10lwXEFh58KTM4ABhMw9SVsN57bLj8fGLgSmPQPzClkHJ7pMBgxzLulOdRuVcC2qeletRTV2eEBgIUG4GBgRSO2zCPgsLqFmqzzvJWWp1G0IcUnAWOXBXZqRDV4KujEb3/iRzENlUMGX0zrWxgxGLDDyysGnRgAYzJVwpJ6kRXGjwIWGH4+HFjJT/SAIR0V2hQblI3UFSqxf2Bh1heH2zCSvIi5M3U4A8yaiMFnASsqxXpgBZ2Ir1/OMUeZ3hJr9YfD71rh1kv3wOI74lOAce6hdk732+DG6VtL1AFloNXDgWGPEQPV68Rt05x7HMiCzoXNZhHDU4FFhTBIYbJdy9ChgtuBreMb2SQnbMDAcDbMVJswxlDAik+ktAHLbmnevoz+jSskbG5VRDsw6hkYacDQbsVADTrR3VJxCDD/PlCnEXk8YAXrnt0rUVv0moOn5QVI6kRYDxnYlEZ1ogNbiqxTYKwDsyVaDwS27qjg5EFQbN35sAGrTnTI5ugMBqzZUwS1GTFXIavE7HoNETBrzRn9A6MEMJITgBk0afLoropemhUYTEpc0Yj92jBOAivsEUuuH3LhTVnEOB0MmYGx0hoAu0bsGpjiqMPwUC49tUicRFDTB+D/Oab9IrYCS2VIUx9VCAb7AaYP3SupvWmGVIgs4XSgKCVw3TpdxFADLKjGsvu4hQ/jz5f7mHQMjLOFXbZ4CKG1QyGTkTUgjHyGiCtsWLB73vnk+Ft+07c3wqLb1FS+Ep9SipLzT5AgbLJDbqf8PmDTLqdYKyAOPH++foh+IOMDS3l8Wf8kNVEMphXXxljV+x3LFFGEagHavs5v+iL/lAvuFRgXgOlKUSnocq4AOHfe2kQkpxPZ6qGv3nkMDNFd/kz/ka4vKzBtcbQBR87rkJKa/cBks0GGAkkIDB6w7TZ+iH6+7Sdr9Q0sM1g75VGi9NjbXDy0AoPjVMZVPnaHE0z27Ifop+bEwvubcCTn5uX7KpB0RBoy9t7np32OwqbK7dWhehAWdl39JQvVvXhtwAypiuiuU/jkUGBIF1kKYppxKeFlP9YKd/fiVQWs0H3LEcqdwAqHhahH0uoaUQPmfs9ePXiDDSMrk3yrIFuAsQlYPtNglDAxpCv7awbYDwycaQ6IXcqdwNjQCbMTmFcQGAuYbT4GzD4jl4HJAcDMXU9hPtMtqY91VQHLHQwbWbjMclqySxaNKFZe4UkyDwDGJmCLUpRiO47kqmgWYLITWOal7BmxRwPLdA2EH5CteBnkgwwaUey8xA87NmDYfv9cYGwARmVgUuTVDAxpYPCLpJBui8uHAEuuUw0wKjh5yB8vYfASM3DdmAPsSPsjgaV9vxZguqsjhcrKgcAIvLms8gJr0IkQJ8mHVmBIvxZhb9agwGQvMNQAS+pEf+fnicC2AAVrKvEFVhQx1n15q0ZMeR3IvNYFNu2Dnn+beibwAnNYqL4hrAKWSlqLBRgYjE0nZpPMTwTG+4HBUgkzAMvrz8Xp4HnOQEGEXmDTpyg6EWyqhBWBwQRsTQEURKw3b+QeL3G1HO5886AbxPRgcwUw0byOooi9wGZjtYqYzH3eYVOdtAITEzA3r1gAhicCA1UAc3cSLYVl/+Q98xrVA4vqsPmjqHsFRuVJ/scBY/KSrnHLiH2JKjQiL7Idfu9sDNZbBcZxOmQPMN4LjJp4Ke0a2Zci3EgByosY9wpMq3JVAANVAEMpASw7gBX9B1UnokC5O2CsrTFsonQ4sKoFqmC9TB7hWCdS7uMrgfF1wNAMjDsBVgiCpz/IZHc7GjTiJcDICsyqESElYEgCq1ue2reGwPJuR3ddHxMwXp82ZNQNkskgtvomSuZpn4D5wHA4MO4TWNRPpOopLiEwhs2c7XarNfG1sqkBQy4Bia6BpRY63xVdJWDOwVEaMDkZmCibPFBoCe9WJfp7S7VWvgM0YlDv2snLe5J2AMNQuUQuZwrrgOVT9ct07dgoNjzN1RkS91aES8CkzzisnHji44DRHI5yNMe+ZWmqgbG7lw3M8/fJfH7HwPSj1YSzp8hXRc1eA0xwnEDbLVQD88dZgWU7NPUAEcMlwNYR1smIyn0sZR8wdh5wryX3amDs5jpy2amaQZfXAJu+FWUOKoTS6dzmJHqjrp0Fb72Feq+DlS9SyPJTdypRotObwn6wuNN50v6IXUsrMDgzquQ6YIj2Q2XDrWbzem55BZJPAHunzcyqTAgMrh0rz36lcu8TXA2MQeEsCGSJUEfDfwO3PnWc9UbM6fKcTqXn2nMAvL2adEDJyWjD2DFi07vYAowvMU0NwEQ7cTvaVwnSvEVuSXRsOnHnYtQCE2IR3/PNuImddZI6R9sTrcOFEisPc7beHLUdsXmfa+M45tldtwCTfrumQASG0iHoyBTvBoa9qY0MMOsbJqkJ5gumfZQOgTmhFsPJBYTylso1VujEsHx4wHLYgFEITGzABCJCvahGt6eDI0sFY0YRphPZEMcNoDuBsQ1YV5LmDGlWpu1xKzB9Vy20Pk7szxSwCRhHwERrlMw8Fl0B8ydoqT0ekmnp4DIwKIe5xBJW+RxjFzB2RB+5B2IqL/QFjJW2C0+fSdo4scWIJYDxAcWMamD+PZciwRlVFzoxC8zJQTAj26sIPQMZAYtmwnmbR2q7AKkGmPhx2PpGSLmh0eXWCzBJAINXcuaE4xe6J+AEMKU0GnyBGmDs60SrHnXObfTGpCbEktFPOBadH7bu31qGD8ErYdqAiZ7Ch57FQlMwpXkd1cCkNNOKVkvJXQMjpuVm4Jmp1C4XZH+w7exhfSdLdQJXNWIw2jDnoCvKpzIdLdgfMK+/3nETuRoYryCiVxkqMdVOohFYKj2YrzavtDoLnGcRW6vJvJY/VmDIdG/7m4UigYIs9XhDcbruBuAAM8ujFdh2hCp6ydf7J6WD3EPSJm9PXKc8Aczn5RB29CkbgaFdJbYCQzFWnk9Y7A2Yk4pgJYjmZLs9k5YlwVQ+2Y5AUuf24QhgXPVOCkIJFMnOjlF3wLgEjMtRmHjBtDCJ5DMgsrOTdAVmXtIQWFEjzgasO2BOk0AdMGUIeqLtu/yzZmAQo2fAwchtA1juJtWRAgYVWHJPJsfiAj2fmMzi7yqQ1QIjS7aZd+Y5bwIGMgBT6jEasMTxNqgvGisQzBl1awIspN9FD3AMDEELmAmYVo8RKy8PWeuNzEVkC4nGfct9bBXzgbGSVC8C41Q9Rmo6PXBIw2b2A9jiGKb9RU3+rz/8QwNG3AqM84DyWySwQyOagFW/LE5Whl+v+nidr19mGKbXLBmK+HC+JDA4OdxSiMXFBtP9wGB73SHA/taeD/c3+3oL/gDYxISVztICMPWoIVANsSPG/hcqxxZfPqcRdwP7pgwT06eFUhgeX0fOzgEVGJhylegKYnwAMLN1qhdlzcR2AIyj9WenAwdRkgKcM081+1qOMGF2T66+eqz6RLU27Hhg8Vm57NQfo0Ee+lmMTcD4AmDUbsUmT4V2eomH2zBliJEHzK9wcZGDfSMSDtmFVVCJZMrRJ4FhfzbxaC9R4o2YWxcwsIlggmqZD2fRngzMTUgVlSJTDKy3wFkTsQ2YpzQNxzxXBc+HaMSKVhlG+egk9JeZ0qSQ1Fyie8ouJaXQUJ/kEzVilSwWRCwyh0z9JX/DFLAbh0VRJJuAGTf78fXAkBfHOIzvrPM3AMLp+GgfsGRNreUB/vrVDqzwB7vail5wTJxlnfbco+D6p22YogEPBPa3Mm51HYkCEO5qK3oBGHtyxYnjCXVgrq8vTGwKxdCoEbNxaMGTKPzFvbMNLgU295DmbwYqBGxN2mCCaF5H3NjGdB0w70DHpATRUBJWSrYRibt9kfxjUxZxYa1rERI0xTlJflwAbHMlcn+SFVerd2CmhzXSievYFW9oJiMItxJjVuq/fa0NE1fTJ/8kT9sAZBino4EYB85DoPCCAw3Xxk8vsj4605P80kgmc9cYjQlMPZ6evgcYKzqxlMhwEyCynVl+nYHfBCv9N9fmWPlcYIB4s0yWkiAfOwHCqhdKwOiJwCDKhJRSigq+Zwi6HNgqYkkjxuZ96kMBYwQixtkIWi+cXa4RHSv2acAk3ALI5fpkdvbYdVlERycipxE/Gphq5HADr+20h4Rc04cAQ77zsATswntmVyemghWm53mJ+4HxHQK2iZjmCN5Vn7sEmEsOeZ9DmbNLNWein2XFfr70ZxC5sHpkCVuPmmFj5VlPS8n1wCalGDYq+c8VarOs/QL7+uUCM3YdJoFdvQTrQxK1AgatKOqbbyXSCGzKus7AkvMwE+n8+IdyBzCKgM0pa2depB5/nnoVmhMbgU11jWWXOsrAOAfsch2zKjwPGM/bsp0nEDGqk4GV2n93AfOiTBswrYp2vU3YCqbu8zwJlkQ7rtmvvNO4wPIuYFEjbhtf5B6dSOEkICbttKWtSDQusMmGxbqtSiM290odJmIUUIR2BmqgH8/26s+xYZOXmFd4RY14Y3SaAKZOUVrreAuwk43YKV6in+hAya1PveC+hJ3qVBCpArZh8/5x2FVVNj8WGA8DjO3A5tfgrERHXWPKMRXnotOReMGNVV1FJ+aBnRY417V+HQoMVAkM95Uw2Cphpz9R1wHTNq5XA5N+gOknFD8ZGBcnBFA3wMTgJhZyvQeph+tsWNTFlt3DMm0SDo0Y5F4RowwwzD9M+PKHhSSXeYn6NBx1HyYvDYjBbpYbJxAqfmICGNStfDcF/cc4HSgnpljdf3SfgGl+ImujNinXqzM0sHwxjNVQTAYBlvyAs3z9U4DJ2hLKZR+RtX3qdwqYohNZ0Yi5fWTkDHGmYYClzqf1gelnZ0pfwJRMfX5rbVkK+wLmdyLmgPmWS00M3QWMUsCkDIwp5XD2CUzizeuJnRDBztnWLbKnGLGUTkyGa2Ekdmkr0T5gnHImYhE7YRLYQcAoB2w2v8kPgPcm9UU9ARNDaYXU88V6AKZJhmLCEKQ02HU1wqn8SbXZCTA2SRhrScT7+2oTqmy2sdjcvwgYkROJBd7VycT2dv5afA7thIIu5gClNdl2Ss6s9Pz3wUHHxfreofm3vcCMJkwDdjuv3Oh1lgywRKCdnHBGHQETCzDEQ93QxU4DQ/w+TzO2JFSTDd5HithuYOWWxMQpVJBuRAzZl0h+fKm2a/FEETsKmBCRfQZfL8DECIxJTMRSOrYnYJNTleelHptjANY+q63ucSsCy4+OLe7GAXcFTKL+S9MAN8N3q55z05oOzUeaZjtOuZ78boABVLzagNVPkroJmMT3qUUBnQDjoYEVOyNt8WJ8n70Gzlo0MhKw4uhloz6L79MLDLqSMGoCZliHK2zYUQdbcrz5zRdR9APMQA5NwC7wEg92N1lp4Zn/0Q8wsaXrW4CNdMWaRLxJIDQUMO04zL5WG6cAc2bdc3/ACvn6joEd8YXizkx4PstRwdiRXqI/67egEzsC5mYAm79XAtjhzv3hwECwnNXX01gg9kc47hBTztwg4RAhO2J0UdROajpdQPojtueLccqtd7xp6gQYR+PBBgPm10jQ/gmcv8EjROxIYFuHr0Un9hZHQfaMKvMLmKkbpD6AuSPqU03AvQMLbqTxmV2NYPJDuCtgpBx8tJwHwSMA20NsbdJcP0h9HboClrBYYMUR6TbVEfqK5m/qnzCU6O2gDoCVXAyom1t6zk3xklfyl77wpZm81+rtU3tF7Apgup7sPpkY2aNysdMBpk60YBGhfXbsQmDgAYyYH+p6OYryjAp2KanAsNvxuEQl6p6ijHWxqdFrY5pWIrwHzZXAiIWrdGIfrXArgPpV2XVb+li3c7xE6PhQedZKB8CY9gBLBM+m20oMTrwR2Bg6kSkkV+u0xB/YCIwuAobUpohBbFdlNOYvS3t+6kwJYzswjOTYH/Egt7/7RBuW9zo+F1jKU7RRPM9LVIFpO8f8rbPgxwNLdD3ucaaOOXtFaQDQNj97wG4+w+QyZBqc/oB50/ZYmSYAGS907uI6DxjWXWMcbJQlb9v38+WpP2CcnmlJEkuY3Da77gVWAkbizi3Cs7xEpt35q5uAZduzof5z67Q8Rinf4wHGvWzPALbsqIXSCMbfF+wpOo1XPJd7BGBcPh+Hk8D8A6GG4xUNycGwwDieRwqlc29MYImU6CDAcjHY6iragGE8XiinovoHFtaW9Vzi9Gy6NowwtIANAoxNG/ji2b/zvfpHro0BjNLAjr+DRe1MC3Xk/jAq7RTjgKWCfmiNKKdW+f73P78uBha1ca83i7GApQSMz0wH/P3Pf/7/WmCz74E0sP7zwVQGRk8BtgTQSOamuk8IczCUWT/PCp0DY+NcDo79ROWD0DevbTZA5kzGzm2YDRhYySYqkorOea3N2xlgZ4nYQV6iCRhYaZyCXKZPDk0ect6GVd4Dt2A7HRhTEVj3JRc9yNwJjK8HZhnLobcmynX6/+DgKw+MzpOvi4AJqwdTaR8zRvCVMGItwHA1MC4Dc0Z3uE4Hdj6cfQDz9mo2AUOVJJ4DLO6tn3erQxewPfv379WI7n00HZpzKDCLvBqBLYkpQebJ7H3jc0HEmoBxZZPiORKmtmpLcvB0/8CyUzwR3gbqdGIPwJBw6VN12t43tOTHroavOQBYci0OAKbeDpPaqZ31hnv2OahM7FA9kVyKQ5K/WGutCjANXfKA8k5plYBhBzBU/fiwOEyWTb3Od+akckw9vehVvEqTqNGuJ1AXTR8eOM/ql4kIYH/cZSLY6h2YZW74oXpCmbV4ZqZj/RlzcMYHdCxPAHbo5u1YB52dmoIIu1vErMBGdDiazlAoPSFIqs0zsvVTaZbjnGIiwfs0YLv1RC42OwMYp5LAYwLjy4GdGjhnh+9B+wWSH4BnAKOHAUuvCB6hEc+9jzM6fzlRZ0HibvhZGtFyIzwQMBkMWAOvsoi17/o4YzME50bioAZYB7szWwSs/OjdC4wztZWozpJ5f3wT9+/ObONVFLGKGSXBmVxnA0NUea4AlpiP1T+vooix2TEJT707Y48zZwwaMm/vDRjtuIqfbQX2HZwreTow+D/PvV36AraH13E68RRgbAJWMmEkPdkwpg8F5m9OpwqNSHd6iTt5HacTT7Fhktsv67d0oArYmOrQ5thbb+8ELzEPzN0Hkc1zdJSY2s+rfDeN93sCME7uNMpnEh/Fy5DsoBuBsWHrSgpYf7UVOuSSc0TseGDQO1SQ2IvDvQFjughYm4hdAYzdXXC9AzuKl0F+mkTsIMc5Bwz+j19guzLAxwPL+xxpYJ04iXQdsBudjhwwzjaRPhcYDQIs0do2CjD+EGDJNg7i4H9hTXQMDwxDAgN9LDAaEhiHP08Ce5oJewow+Rxg6BiYJIBJqa/e25/0MGA0HrDYe5RPAobRgCFSjmltKg8ERkMAkwww6R4Yfx6wWZS09qn0GnWiEI8GhmGAgSQGll4kkWcCo2GAsXO8QBFYT9cnAlsmtomvHMY4RIA+GFggX/wC6wkYe8C0IuYnAsMQwFib+Us0xslFHwkM6ln3L7BuVaLiaYEJr0rsFJj+1fm1YUMBI9ALrCdgUgJGY5xc9AIbCxh/DjAuAsPnAZOOgS2pQ4zM66OATcTAI1uwjwLGRKLxkoEE7MMkjEgbnricMPAC6w0Y6+FXupvj8cByWoXvB5Y854iH0YgvMNAnA5O+VWLqyHSMoxFfYMuxsoMI2Ae1aqeAzdUW+URg0jsw9W6FRxKwF9hAWY6jgaF/YDxyGvETgcnwAvYxe5xfYGMC49E14oHAMAIwGV3AssBQZeNkWGAjCZgUl/9mAbsCmIwPrEFnyhDAeHQB4/La3ytgFwCTwYGhKVSTYYHJ4MDQ5kgOCwxjAbN8fw5Gxlxrt88GJmMDQ/JFWdUoowCTwXmxST/Mv7mnxflcYCPzQvZlOVOGgYDx+pUH5GU97padw8UvDzyPPt3EecKG45U5eS64ybyvOBQwGc0v1PVD/kJWwDAWsKEvY8srRPiuTQQvsDYFwfICGwvsIf0QeIHdF7O1CBi9wG4HViU0DQ7KC+ywrE6Di/gCux2YvMCG0omVAF4bdjew2o94gd0LDC+wsWzYC2wwYNVC+jodQ2nEF9jNwBo+4wV2p0ps+Ai8wG4i1qTfiKg2A/wC26sUeUfRsmHw5wtsJ7E93SstivQFtg8Zlp6cdpX6ArvclmEPL7zALhezPWFBLewX2L0+5uvWj+Vi1ivTF9itwOQFNpRKxAtsLGDyAhtKJ8oLbCgJwwtsoLitNT/yAtsnJs1vak0Yv8B267XaKuS837GN1wtsv6tXufbcWjx7gR0G7Ap3/gV2HDDUvgMvsGGA8QvsfmBXCtgLbF9EVbv++zetv8D2AsOJr3+BnaARr+X1AtsLrFaBygvsVmJXOogvsBti7BfYh/F6gV3o/x8yheoFNhavF9h1vI4ZQvUCu4wXXmBD8TpoytsL7CL/8Ki5ly+wi3gdNUbxBTaUQnyBDeUhvsAu04jyAhuKmLzAPvZ6gb3A3sv1OfACeyXsvV5g7/UCe68X2AvsvV5g7/UCe4G91wvsvV5gL7D3eoG91wvsBfZeL7D3eoG9wN7rBfZeL7DnA/svfacONHhmjCoAAAAASUVORK5CYII='
# Generate the Mask for the world coastline
f1 = open("world.png", "wb")
f1.write(codecs.decode(world,'base64'))
f1.close()
world = cv2.imread("world.png")
world = cv2.cvtColor( world, cv2.COLOR_RGB2GRAY )
# plt.imshow(sing)

Moment of truth. Let us estimate the Fractal dimension of the coastlines of the world and look at its plots:

In [ ]:
Z = 1.0 - world/255
#sizes = 128, 64, 32
sizes = 32, 16, 8
xmin, xmax = 0, Z.shape[1]
ymin, ymax = 0, Z.shape[0]
fig = plt.figure(figsize=(40, 27))
for i, size in enumerate(sizes):
    ax = plt.subplot(1, len(sizes), i+1, frameon=False)
    ax.imshow(1-Z, plt.cm.gray, interpolation="bicubic", vmin=0, vmax=1,
              extent=[xmin, xmax, ymin, ymax], origin="upper")
    ax.set_xticks([])
    ax.set_yticks([])
    for y in range(Z.shape[0]//size+1):
        for x in range(Z.shape[1]//size+1):
            s = (Z[y*size:(y+1)*size, x*size:(x+1)*size] > 0.25).sum()
            if s > 0 and s < size*size:
                rect = patches.Rectangle(
                    (x*size, Z.shape[0]-1-(y+1)*size),
                    width=size, height=size,
                    linewidth=.5, edgecolor='.25',
                    facecolor='.75', alpha=.25,color='darkblue')
                ax.add_patch(rect)

plt.tight_layout()
# plt.savefig("fractal-dimension.png")
plt.show()
print(fractal_dimension(Z, threshold=0.25))

Pretty interesting result. We get a Fractal Dimension value that is rather close to the other three smaller coastlines. 

# 4. Fractal Dimension used in feature engineering

## *TO BE CONTINUED. NEED TO SLEEP*